In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import random
import os
from collections import deque

# Fix random state for consistency
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Simplified network architecture - removed CNN for speed
class FastDQNetwork(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=64):
        super(FastDQNetwork, self).__init__()
        
        # Simpler LSTM architecture
        self.lstm = nn.LSTM(
            input_size=1,  # Single feature input
            hidden_size=hidden_size,
            num_layers=1,  # Reduced from 2 to 1
            batch_first=True
        )
        
        # Simpler Q-value prediction layers
        self.q_layers = nn.Sequential(
            nn.Linear(hidden_size, action_size)  # Direct mapping
        )
    
    def forward(self, state):
        # Handle different input shapes efficiently
        if isinstance(state, np.ndarray):
            state = torch.FloatTensor(state)
            
        batch_size = state.size(0) if state.dim() > 1 else 1
        seq_len = state.size(-1) if state.dim() > 1 else state.size(0)
        
        # Reshape to [batch_size, seq_len, features]
        if state.dim() == 1:
            # Single sample: [seq_len] -> [1, seq_len, 1]
            x = state.view(1, seq_len, 1)
        elif state.dim() == 2:
            # Batch: [batch_size, seq_len] -> [batch_size, seq_len, 1]
            x = state.view(batch_size, seq_len, 1)
        else:
            # Already in correct shape
            x = state
            
        # LSTM
        lstm_out, _ = self.lstm(x)
        last_out = lstm_out[:, -1, :]
        
        # Get Q-values for each action
        q_values = self.q_layers(last_out)
        
        return q_values

# Optimized Replay Buffer with numpy arrays instead of deque operations
class FastReplayBuffer:
    def __init__(self, capacity=10000, state_size=50):
        self.capacity = capacity
        self.state_size = state_size
        
        # Pre-allocate memory for all experiences
        self.states = np.zeros((capacity, state_size), dtype=np.float32)
        self.next_states = np.zeros((capacity, state_size), dtype=np.float32)
        self.actions = np.zeros(capacity, dtype=np.int64)
        self.rewards = np.zeros(capacity, dtype=np.float32)
        self.dones = np.zeros(capacity, dtype=np.float32)
        
        self.position = 0
        self.size = 0
    
    def add(self, state, action, reward, next_state, done):
        # Store experience
        self.states[self.position] = state
        self.actions[self.position] = action
        self.rewards[self.position] = reward
        self.next_states[self.position] = next_state
        self.dones[self.position] = float(done)
        
        # Update position and size
        self.position = (self.position + 1) % self.capacity
        self.size = min(self.size + 1, self.capacity)
    
    def sample(self, batch_size):
        # Sample random indices
        indices = np.random.choice(self.size, batch_size, replace=False)
        
        # Return experiences as tensors
        return (
            torch.FloatTensor(self.states[indices]),
            torch.LongTensor(self.actions[indices]),
            torch.FloatTensor(self.rewards[indices]),
            torch.FloatTensor(self.next_states[indices]),
            torch.FloatTensor(self.dones[indices])
        )
    
    def __len__(self):
        return self.size

# NEW: System model for simulation
class SystemDynamicsModel:
    def __init__(self, inertia=1.0, damping=0.1, dt=0.1):
        """Simple dynamic system model (e.g., motor or physical system)
        
        Args:
            inertia: System inertia coefficient
            damping: System damping coefficient
            dt: Time step for discrete simulation
        """
        self.inertia = inertia
        self.damping = damping
        self.dt = dt
        
        # System state
        self.position = 0.0
        self.velocity = 0.0
        
    def reset(self, position=0.0, velocity=0.0):
        """Reset system to initial state"""
        self.position = position
        self.velocity = velocity
        return self.position
        
    def step(self, control_force):
        """Apply control force and update system state
        
        Simple second-order dynamics model: m*a + b*v = F
        Where:
            m = inertia
            b = damping coefficient
            F = control force
        """
        # Calculate acceleration from control force and current state
        acceleration = (control_force - self.damping * self.velocity) / self.inertia
        
        # Update velocity and position using simple Euler integration
        self.velocity += acceleration * self.dt
        self.position += self.velocity * self.dt
        
        return self.position

# NEW: Sensor interface for real-world applications
class SensorInterface:
    def __init__(self, use_simulation=True, sensor_noise=0.01):
        """Sensor interface that can read from simulation or real hardware
        
        Args:
            use_simulation: If True, use simulated data. If False, read from real sensors.
            sensor_noise: Amount of noise to add to simulated sensor readings
        """
        self.use_simulation = use_simulation
        self.sensor_noise = sensor_noise
        
        # If not using simulation, set up hardware communication
        if not use_simulation:
            try:
                # This would be replaced with actual hardware setup code
                # e.g., import serial, GPIO setup, etc.
                print("Setting up hardware communication...")
                # self.serial_port = serial.Serial('/dev/ttyUSB0', 9600)
            except Exception as e:
                print(f"Hardware setup failed: {e}")
                print("Falling back to simulation mode")
                self.use_simulation = True
    
    def read_position(self, simulated_position=None):
        """Read current position from sensor or simulation"""
        if self.use_simulation:
            # Add noise to simulated position
            return simulated_position + np.random.normal(0, self.sensor_noise)
        else:
            # This would be replaced with actual sensor reading code
            # e.g., self.serial_port.write(b'READ_POS\n')
            # return float(self.serial_port.readline())
            return 0.0  # Placeholder

# NEW: Actuator interface for real-world applications
class ActuatorInterface:
    def __init__(self, use_simulation=True, control_min=-1.0, control_max=1.0):
        """Actuator interface that can send commands to simulation or real hardware
        
        Args:
            use_simulation: If True, return values for simulation. If False, send to real actuators.
            control_min: Minimum control signal value
            control_max: Maximum control signal value
        """
        self.use_simulation = use_simulation
        self.control_min = control_min
        self.control_max = control_max
        
        # If not using simulation, set up hardware communication
        if not use_simulation:
            try:
                # This would be replaced with actual hardware setup code
                print("Setting up actuator communication...")
                # self.serial_port = serial.Serial('/dev/ttyUSB1', 9600)
            except Exception as e:
                print(f"Actuator setup failed: {e}")
                print("Falling back to simulation mode")
                self.use_simulation = True
    
    def send_control(self, control_signal):
        """Send control signal to actuator or return for simulation"""
        # Clip control signal to valid range
        control_signal = np.clip(control_signal, self.control_min, self.control_max)
        
        if self.use_simulation:
            return control_signal
        else:
            # This would be replaced with actual actuator control code
            # e.g., self.serial_port.write(f'SET_PWM {control_signal}\n')
            print(f"Sending control signal: {control_signal}")
            return control_signal

# Modified: RL Controller Environment (previously TrajectoryEnvironment)
class RLControlEnvironment:
    def __init__(self, reference_trajectory, window_size=50, control_discretization=20,
                 use_simulation=True, control_effort_weight=0.1):
        """Environment for the RL Controller
        
        Args:
            reference_trajectory: Target trajectory to follow
            window_size: Size of state window (recent positions and setpoints)
            control_discretization: Number of discrete control actions
            use_simulation: If True, use simulated system. If False, use real hardware.
            control_effort_weight: Weight of control effort term in reward (λ)
        """
        self.reference_trajectory = reference_trajectory
        self.window_size = window_size
        self.use_simulation = use_simulation
        self.control_effort_weight = control_effort_weight
        
        # Create discrete action space (control signals)
        self.control_range = [-1.0, 1.0]  # Normalized control range
        self.action_values = np.linspace(
            self.control_range[0], 
            self.control_range[1], 
            control_discretization
        )
        self.action_size = len(self.action_values)
        
        # State is the window of previous values and setpoints
        # [positions, setpoints] concatenated
        self.state_size = window_size
        
        # Initialize system model, sensors, and actuators
        self.system = SystemDynamicsModel()
        self.sensor = SensorInterface(use_simulation=use_simulation)
        self.actuator = ActuatorInterface(use_simulation=use_simulation)
        
        # Current position in the reference trajectory
        self.current_idx = window_size
        self.max_idx = len(reference_trajectory) - 1
        
        # History of positions and controls
        self.position_history = np.zeros(window_size)
        self.control_history = np.zeros(window_size)
        
    def reset(self):
        """Reset environment to starting point"""
        self.current_idx = self.window_size
        self.system.reset()
        
        # Reset history
        self.position_history = np.zeros(self.window_size)
        self.control_history = np.zeros(self.window_size)
        
        # Initial state combines position history and setpoint trajectory
        # For simplicity, we'll just use the position history as the state
        state = self.position_history.reshape(-1)
        return state
    
    def step(self, action):
        """Take control action and return new state, reward, done"""
        # Get the control signal corresponding to this action
        control_signal = self.action_values[action]
        
        # Send control signal to actuator
        applied_control = self.actuator.send_control(control_signal)
        
        # Apply control to system (simulation or real)
        if self.use_simulation:
            # Update system with control input
            new_position = self.system.step(applied_control)
            
            # Read position from sensor (with noise)
            measured_position = self.sensor.read_position(new_position)
        else:
            # In real-world mode, just read the sensor after applying control
            # The control signal has already been sent by the actuator interface
            measured_position = self.sensor.read_position()
        
        # Get current setpoint from reference trajectory
        current_setpoint = self.reference_trajectory[self.current_idx][0]
        
        # Calculate tracking error
        tracking_error = abs(measured_position - current_setpoint)
        
        # Calculate control effort penalty
        control_effort = abs(control_signal)
        
        # Calculate reward: -(tracking error) - λ(control effort)
        reward = -tracking_error - self.control_effort_weight * control_effort
        
        # Update histories
        self.position_history = np.append(self.position_history[1:], measured_position)
        self.control_history = np.append(self.control_history[1:], control_signal)
        
        # Move to next setpoint
        self.current_idx += 1
        done = self.current_idx >= self.max_idx
        
        # Get new state
        if not done:
            # State is the history of positions
            next_state = self.position_history.reshape(-1)
        else:
            next_state = np.zeros(self.window_size)  # Default state when done
            
        return next_state, reward, done
    
    def get_current_setpoint(self):
        """Get the current target setpoint"""
        if self.current_idx < len(self.reference_trajectory):
            return self.reference_trajectory[self.current_idx][0]
        return None

# Optimized DQN Agent - Minimal changes for control application
class FastDQNAgent:
    def __init__(self, state_size, action_size, hidden_size=64, device='cpu'):
        self.state_size = state_size
        self.action_size = action_size
        self.device = device
        
        # Hyperparameters - increased learning rate and batch size for faster convergence
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.97  # Faster decay
        self.learning_rate = 0.003  # Increased from 0.001
        self.batch_size = 128  # Increased from 64
        self.update_target_every = 5  # More frequent updates
        
        # Networks
        self.q_network = FastDQNetwork(state_size, action_size, hidden_size).to(device)
        self.target_network = FastDQNetwork(state_size, action_size, hidden_size).to(device)
        self.target_network.load_state_dict(self.q_network.state_dict())
        self.target_network.eval()  # target network in eval mode
        
        # Optimizer with higher learning rate
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=self.learning_rate)
        
        # Memory
        self.memory = FastReplayBuffer(capacity=10000, state_size=state_size)
        
        # Tracking
        self.episode_count = 0
        
        # Create evaluation mode flag to avoid unnecessary computation
        self.training_mode = True
    
    def select_action(self, state, training=True):
        """Select action using epsilon-greedy policy"""
        self.training_mode = training
        
        if training and random.random() < self.epsilon:
            # Exploration: random action
            return random.randrange(self.action_size)
        
        # Exploitation: best action from Q-network
        with torch.no_grad():  # Ensure we don't compute gradients
            # Convert to tensor if it's a numpy array
            if isinstance(state, np.ndarray):
                state = torch.FloatTensor(state)
            
            # Add batch dimension if needed
            if state.dim() == 1:
                state = state.unsqueeze(0)  # [L] -> [1, L]
                
            # Add channel dimension if needed
            if state.dim() == 2 and state.size(1) != 1:
                state = state.unsqueeze(2)  # [B, L] -> [B, L, 1]
                
            state = state.to(self.device)
            q_values = self.q_network(state)
            return torch.argmax(q_values).item()
    
    def train(self):
        """Train on a batch from replay memory"""
        if len(self.memory) < self.batch_size:
            return 0  # Not enough samples
        
        # Sample batch
        states, actions, rewards, next_states, dones = self.memory.sample(self.batch_size)
        
        # Move to device
        states = states.to(self.device)
        next_states = next_states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        dones = dones.to(self.device)
        
        # Get current Q values - more efficient indexing
        current_q = self.q_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Get next Q values - more efficient with no_grad
        with torch.no_grad():
            # Double DQN
            next_actions = torch.argmax(self.q_network(next_states), dim=1)
            next_q = self.target_network(next_states).gather(1, next_actions.unsqueeze(1)).squeeze(1)
            
            # Apply terminal state masking and calculate targets
            target_q = rewards + (1 - dones) * self.gamma * next_q
        
        # Calculate loss and optimize
        loss = nn.MSELoss()(current_q, target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        # Gradient clipping to prevent exploding gradients
        nn.utils.clip_grad_norm_(self.q_network.parameters(), 1.0)
        self.optimizer.step()
        
        return loss.item()
    
    def update_target_network(self):
        """Update target network weights with soft update instead of hard copy"""
        tau = 0.1  # Soft update parameter
        for target_param, param in zip(self.target_network.parameters(), self.q_network.parameters()):
            target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)
    
    def decay_epsilon(self):
        """Decay exploration rate"""
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Get data - similar to original code
def get_data(file_path="Trajectory2.csv"):
    try:
        df = pd.read_csv(file_path)
        
        # Check columns
        if 'x_Traject' in df.columns:
            data = df['x_Traject'].values.reshape(-1, 1)
        else:
            # No header? Let's try again
            df = pd.read_csv(file_path, header=None)
            data = df.values
            
    except Exception as e:
        print(f"Couldn't load data: {e}")
        print("Using synthetic data instead")
        
        # Make fake trajectory data - temperature setpoint scenario
        t = np.linspace(0, 1, 7201)
        data = np.zeros((len(t), 1))
        
        # Let's make something interesting with segments
        data[:1000] = 45.0  # Starting temperature
        
        # Ramp up
        for i in range(1000, 3000):
            progress = (i - 1000) / 2000
            data[i] = 45.0 + progress * (70.0 - 45.0)
            
        # Hold temp
        data[3000:5000] = 70.0
        
        # Ramp down
        for i in range(5000, 6000):
            progress = (i - 5000) / 1000
            data[i] = 70.0 - progress * (70.0 - 50.0)
            
        # Final hold
        data[6000:] = 50.0
        
        # Add a bit of noise to make it realistic
        data += np.random.normal(0, 0.1, size=data.shape)
        
    return data.astype(np.float32)  # Cast to float32 for faster tensor operations

# Modified: Training function for the RL controller
def train_rl_controller(env, agent, episodes=100):
    """Train the controller agent for the specified number of episodes"""
    rewards_history = []
    loss_history = []
    tracking_errors = []
    control_efforts = []
    
    # Use mini-batches for faster training
    for episode in range(episodes):
        # Reset environment
        state = env.reset()
        total_reward = 0
        episode_losses = []
        episode_errors = []
        episode_efforts = []
        done = False
        
        # Initialize batch tracking
        steps = 0
        update_frequency = 4  # Update every N steps instead of every step
        
        while not done:
            # Select and take action
            action = agent.select_action(state)
            next_state, reward, done = env.step(action)
            
            # Calculate tracking error and control effort for logging
            current_position = env.position_history[-1]
            current_setpoint = env.get_current_setpoint()
            control_signal = env.action_values[action]
            
            tracking_error = abs(current_position - current_setpoint)
            control_effort = abs(control_signal)
            
            episode_errors.append(tracking_error)
            episode_efforts.append(control_effort)
            
            # Store in memory
            agent.memory.add(state, action, reward, next_state, done)
            
            # Move to next state
            state = next_state
            total_reward += reward
            steps += 1
            
            # Train the network less frequently
            if steps % update_frequency == 0:
                loss = agent.train()
                if loss > 0:
                    episode_losses.append(loss)
        
        # Update target network periodically using soft updates
        agent.update_target_network()
        
        # Decay exploration rate
        agent.decay_epsilon()
        
        # Track metrics
        rewards_history.append(total_reward)
        if episode_losses:
            loss_history.append(np.mean(episode_losses))
        else:
            loss_history.append(0)
        tracking_errors.append(np.mean(episode_errors))
        control_efforts.append(np.mean(episode_efforts))
        
        # Print progress
        if (episode + 1) % 5 == 0:  # Less frequent logging
            print(f"Episode {episode+1}/{episodes} - Reward: {total_reward:.4f}, "
                  f"Loss: {np.mean(episode_losses) if episode_losses else 0:.4f}, Epsilon: {agent.epsilon:.4f}, "
                  f"Mean Error: {np.mean(episode_errors):.4f}, Mean Control: {np.mean(episode_efforts):.4f}")
    
    return {
        'rewards': rewards_history,
        'losses': loss_history,
        'tracking_errors': tracking_errors,
        'control_efforts': control_efforts
    }

# Modified: Evaluate the trained controller
def evaluate_controller(env, agent, scaler=None):
    """Evaluate the control performance on the reference trajectory"""
    state = env.reset()
    done = False
    
    # Create arrays to store results
    positions = []
    setpoints = []
    control_signals = []
    
    # Set evaluation mode
    agent.training_mode = False
    
    with torch.no_grad():  # Disable gradient calculation for evaluation
        while not done:
            # Select best action (no exploration)
            action = agent.select_action(state, training=False)
            
            # Get setpoint before step
            current_setpoint = env.get_current_setpoint()
            
            # Take step
            next_state, _, done = env.step(action)
            
            # Record results
            positions.append(env.position_history[-1])  # Last position
            setpoints.append(current_setpoint)
            control_signals.append(env.action_values[action])
            
            # Update state
            state = next_state
    
    # Convert to numpy arrays
    positions = np.array(positions)
    setpoints = np.array(setpoints)
    control_signals = np.array(control_signals)
    
    # Calculate performance metrics
    tracking_error = np.mean(np.abs(positions - setpoints))
    control_effort = np.mean(np.abs(control_signals))
    
    print(f"Evaluation Results:")
    print(f"Mean Tracking Error: {tracking_error:.4f}")
    print(f"Mean Control Effort: {control_effort:.4f}")
    
    # Rescale if scaler is provided
    if scaler is not None:
        positions = scaler.inverse_transform(positions.reshape(-1, 1)).flatten()
        setpoints = scaler.inverse_transform(np.array(setpoints).reshape(-1, 1)).flatten()
    
    return {
        'positions': positions,
        'setpoints': setpoints,
        'control_signals': control_signals,
        'tracking_error': tracking_error,
        'control_effort': control_effort
    }

# Create enhanced visualizations for controller performance
def create_controller_visualizations(results, training_metrics=None):
    """Create visualizations for the controller performance"""
    # Set up the figure with a custom layout
    plt.figure(figsize=(15, 12))
    grid = gridspec.GridSpec(3, 1, height_ratios=[2, 1, 1])
    
    # Extract data from results
    positions = results['positions']
    setpoints = results['setpoints']
    control_signals = results['control_signals']
    
    # First plot - control performance (position vs setpoint)
    perf_ax = plt.subplot(grid[0])
    time_steps = np.arange(len(positions))
    
    perf_ax.plot(time_steps, setpoints, 'r--', linewidth=2, label='Setpoint')
    perf_ax.plot(time_steps, positions, 'b-', linewidth=1.5, label='System Position')
    
    perf_ax.set_title('Control System Performance', fontsize=14)
    perf_ax.set_xlabel('Time Steps', fontsize=12)
    perf_ax.set_ylabel('Position', fontsize=12)
    perf_ax.grid(True)
    perf_ax.legend()
    
    # Second plot - control signals
    control_ax = plt.subplot(grid[1])
    control_ax.plot(time_steps, control_signals, 'g-', linewidth=1.5)
    control_ax.set_title('Control Signals', fontsize=14)
    control_ax.set_xlabel('Time Steps', fontsize=12)
    control_ax.set_ylabel('Control Signal', fontsize=12)
    control_ax.grid(True)
    
    # Third plot - tracking error
    error_ax = plt.subplot(grid[2])
    tracking_error = np.abs(positions - setpoints)
    error_ax.plot(time_steps, tracking_error, 'r-', linewidth=1.5)
    error_ax.set_title('Tracking Error', fontsize=14)
    error_ax.set_xlabel('Time Steps', fontsize=12)
    error_ax.set_ylabel('Error', fontsize=12)
    error_ax.grid(True)
    
    # If training metrics are provided, create additional plots
    if training_metrics:
        plt.figure(figsize=(15, 10))
        grid2 = gridspec.GridSpec(2, 2)
        
        # Plot rewards
        rewards_ax = plt.subplot(grid2[0, 0])
        rewards_ax.plot(training_metrics['rewards'], 'b-')
        rewards_ax.set_title('Training Rewards', fontsize=14)
        rewards_ax.set_xlabel('Episode', fontsize=12)
        rewards_ax.set_ylabel('Total Reward', fontsize=12)
        rewards_ax.grid(True)
        
        # Plot losses
        losses_ax = plt.subplot(grid2[0, 1])
        losses_ax.plot(training_metrics['losses'], 'g-')
        losses_ax.set_title('Training Loss', fontsize=14)
        losses_ax.set_xlabel('Episode', fontsize=12)
        losses_ax.set_ylabel('Loss', fontsize=12)
        losses_ax.grid(True)
        
        # Plot tracking errors
        errors_ax = plt.subplot(grid2[1, 0])
        errors_ax.plot(training_metrics['tracking_errors'], 'r-')
        errors_ax.set_title('Training Tracking Errors', fontsize=14)
        errors_ax.set_xlabel('Episode', fontsize=12)
        errors_ax.set_ylabel('Mean Tracking Error', fontsize=12)
        errors_ax.grid(True)
        
        # Plot control efforts
        efforts_ax = plt.subplot(grid2[1, 1])
        efforts_ax.plot(training_metrics['control_efforts'], 'orange')
        efforts_ax.set_title('Training Control Efforts', fontsize=14)
        efforts_ax.set_xlabel('Episode', fontsize=12)
        efforts_ax.set_ylabel('Mean Control Effort', fontsize=12)
        efforts_ax.grid(True)
    
    # Make sure everything fits nicely
    plt.tight_layout()
    
    # Save and display
    plt.savefig('controller_visualization.png', dpi=300, bbox_inches='tight')
    
    print("Controller visualization saved as 'controller_visualization.png'")

def main():
    print("Starting RL Controller implementation...")
    
    # Configure torch for faster execution
    torch.set_num_threads(4)  # Limit number of threads
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True  # Optimize CUDA operations
    
    # Get reference trajectory data
    reference_data = get_data()
    print(f"Got reference trajectory with shape: {reference_data.shape}")
    
    # Normalize between 0-1
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(reference_data)
    
    # Setup the environment
    window_size = 50
    control_discretization = 20  # Number of discrete control actions
    
    # Create environment with control-oriented reward
    env = RLControlEnvironment(
        reference_trajectory=normalized_data,
        window_size=window_size,
        control_discretization=control_discretization,
        use_simulation=True,  # Use simulation mode
        control_effort_weight=0.1  # Weight for control effort penalty
    )
    
    # Create the agent
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    agent = FastDQNAgent(
        state_size=window_size,
        action_size=control_discretization,
        hidden_size=64,
        device=device
    )
    
    # Train the controller agent
    print(f"Training RL controller on {device}...")
    training_metrics = train_rl_controller(env, agent, episodes=100)
    
    # Save the trained controller
    torch.save({
        'q_network': agent.q_network.state_dict(),
        'target_network': agent.target_network.state_dict(),
        'rewards': training_metrics['rewards'],
        'losses': training_metrics['losses'],
        'tracking_errors': training_metrics['tracking_errors'],
        'control_efforts': training_metrics['control_efforts'],
        'scaler': scaler
    }, 'rl_controller_model.pth')
    
    print("Training complete! Evaluating controller and creating visualizations...")
    
def main():
    print("Starting RL Controller implementation...")
    
    # Configure torch for faster execution
    torch.set_num_threads(4)  # Limit number of threads
    if torch.cuda.is_available():
        torch.backends.cudnn.benchmark = True  # Optimize CUDA operations
    
    # Get reference trajectory data
    reference_data = get_data()
    print(f"Got reference trajectory with shape: {reference_data.shape}")
    
    # Normalize between 0-1
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(reference_data)
    
    # Setup the environment
    window_size = 50
    control_discretization = 20  # Number of discrete control actions
    
    # Create environment with control-oriented reward
    env = RLControlEnvironment(
        reference_trajectory=normalized_data,
        window_size=window_size,
        control_discretization=control_discretization,
        use_simulation=True,  # Use simulation mode
        control_effort_weight=0.1  # Weight for control effort penalty
    )
    
    # Create the agent
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    agent = FastDQNAgent(
        state_size=window_size,
        action_size=control_discretization,
        hidden_size=64,
        device=device
    )
    
    # Train the controller agent
    print(f"Training RL controller on {device}...")
    training_metrics = train_rl_controller(env, agent, episodes=100)
    
    # Save the trained controller
    torch.save({
        'q_network': agent.q_network.state_dict(),
        'target_network': agent.target_network.state_dict(),
        'rewards': training_metrics['rewards'],
        'losses': training_metrics['losses'],
        'tracking_errors': training_metrics['tracking_errors'],
        'control_efforts': training_metrics['control_efforts'],
        'scaler': scaler
    }, 'rl_controller_model.pth')
    
    print("Training complete! Evaluating controller and creating visualizations...")
    
    # Evaluate the trained controller
    evaluation_results = evaluate_controller(env, agent, scaler)
    
    # Create visualizations for controller performance
    create_controller_visualizations(evaluation_results, training_metrics)
    
    print("RL Controller implementation completed successfully!")

# Run the main function when the script is executed
if __name__ == "__main__":
    main()

Starting RL Controller implementation...
Got reference trajectory with shape: (7201, 1)
Training RL controller on cpu...
